In [1]:
!pip install segmentation_models_pytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from collections import Counter
import shutil
from sklearn.cluster import KMeans
import cv2
import glob
from tqdm import tqdm
from timeit import default_timer as timer
import torchvision.models as models
import torch.optim as optim
import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from numpy import linalg as LA
from torch import optim, nn
from torch.utils.data.dataset import Dataset

In [ ]:
import os
import shutil

def merge_folders(folder1, folder2, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Copy images from folder1 to destination_folder
    for filename in os.listdir(folder1):
        source_path = os.path.join(folder1, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.copy2(source_path, destination_path)

    # Copy images from folder2 to destination_folder
    for filename in os.listdir(folder2):
        source_path = os.path.join(folder2, filename)
        destination_path = os.path.join(destination_folder, filename)
        
        # Handle duplicate filenames by renaming them
        if os.path.exists(destination_path):
            base, extension = os.path.splitext(filename)
            new_filename = f"{base}_copy{extension}"
            destination_path = os.path.join(destination_folder, new_filename)
        
        shutil.copy2(source_path, destination_path)

# Replace with the paths of your source folders and destination folder
folder1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'
folder2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'
destination_folder = '/kaggle/working/train_image'

merge_folders(folder1, folder2, destination_folder)

print("Images merged successfully!")


In [2]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

NUM_WORKERS = os.cpu_count()

image_size = (256, 256)
height, width = image_size
batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

class SkinDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_names = [file for file in os.listdir(image_dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.image_dir, image_name)
        
        # Construct the corresponding mask path
        mask_name = image_name.replace('.jpg', '_segmentation.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Load the image and mask
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Ensure mask is binary (0 and 1) if needed
        mask = (mask > 0).astype('float32')

        # Apply transformations if provided
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        return image, mask

# Define transformations using Albumentations
transform = A.Compose([
    A.Resize(image_size[0], image_size[1]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

image_dir = '/kaggle/working/train_image'
mask_dir = '/kaggle/input/ham10000-lesion-segmentations/HAM10000_segmentations_lesion_tschandl'

# Load dataset
dataset = SkinDataset(image_dir, mask_dir, transform=transform)

# Split dataset into training and testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, padding=padding, groups=in_channels)
        self.bn = nn.BatchNorm2d(in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.bn(x)
        x = self.pointwise(x)
        return self.relu(x)

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, extra_concat_channels=0):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(mid_channels + extra_concat_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x, extra_concat=None):
        x = self.relu(self.conv1(x))
        if extra_concat is not None:
            x = torch.cat([x, extra_concat], dim=1)
        x = self.relu(self.conv2(x))
        return x

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels, skip_in_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.depthwise_conv = DepthwiseSeparableConv(skip_in_channels, skip_in_channels)
        self.attention = nn.Sequential(
            nn.Conv2d(skip_in_channels, skip_in_channels // 8, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(skip_in_channels // 8, skip_in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.conv = DoubleConv(in_channels // 2 + skip_in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x2 = self.depthwise_conv(x2)
        attn = self.attention(x2)
        x2 = x2 * attn  # Apply attention to skip connection
        x = torch.cat([x1, x2], dim=1)
        return self.conv(x)

class ASPPBottleneck(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.aspp1 = nn.Conv2d(in_channels, in_channels // 4, kernel_size=1)
        self.aspp2 = nn.Conv2d(in_channels, in_channels // 4, kernel_size=3, padding=1, dilation=1)
        self.aspp3 = nn.Conv2d(in_channels, in_channels // 4, kernel_size=3, padding=2, dilation=2)
        self.aspp4 = nn.Conv2d(in_channels, in_channels // 4, kernel_size=3, padding=3, dilation=3)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        aspp1 = self.aspp1(x)
        aspp2 = self.aspp2(x)
        aspp3 = self.aspp3(x)
        aspp4 = self.aspp4(x)
        x = torch.cat([aspp1, aspp2, aspp3, aspp4], dim=1)
        return self.relu(self.conv(x))

class CustomUNetWithEfficientNetB3(nn.Module):
    def __init__(self, num_classes=1, pretrained=True):
        super().__init__()
        self.encoder = smp.encoders.get_encoder(
            "efficientnet-b3",
            weights="imagenet" if pretrained else None,
        )
        encoder_channels = self.encoder.out_channels

        self.bottleneck = ASPPBottleneck(encoder_channels[-1], 1024)

        self.up_conv1 = UpSample(1024, encoder_channels[-2], encoder_channels[-2])
        self.up_conv2 = UpSample(encoder_channels[-2], encoder_channels[-3], encoder_channels[-3])
        self.up_conv3 = UpSample(encoder_channels[-3], encoder_channels[-4], encoder_channels[-4])
        self.up_conv4 = UpSample(encoder_channels[-4], encoder_channels[-5], encoder_channels[-5])

        self.final_upsample = nn.ConvTranspose2d(encoder_channels[-5], encoder_channels[-5], kernel_size=2, stride=2)
        self.final_conv = nn.Conv2d(encoder_channels[-5], num_classes, kernel_size=1)
        self.aux_conv1 = nn.Conv2d(encoder_channels[-2], num_classes, kernel_size=1)
        self.aux_conv2 = nn.Conv2d(encoder_channels[-3], num_classes, kernel_size=1)

    def forward(self, x):
        features = self.encoder(x)
        b = self.bottleneck(features[-1])
        up1 = self.up_conv1(b, features[-2])
        up2 = self.up_conv2(up1, features[-3])
        up3 = self.up_conv3(up2, features[-4])
        up4 = self.up_conv4(up3, features[-5])
        up4 = self.final_upsample(up4)
        final = self.final_conv(up4)

        if self.training:
            aux1 = self.aux_conv1(up1)
            aux2 = self.aux_conv2(up2)
            aux1 = nn.functional.interpolate(aux1, size=x.shape[2:], mode='bilinear', align_corners=False)
            aux2 = nn.functional.interpolate(aux2, size=x.shape[2:], mode='bilinear', align_corners=False)
            return final, aux1, aux2
        return final

# Example usage:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomUNetWithEfficientNetB3(num_classes=1, pretrained=True).to(device)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Display a detailed summary using torchinfo (optional)
from torchinfo import summary

in_channels=3
summary(model, input_size=(1, in_channels, 256, 256))

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define Dice + BCE Loss for multiple outputs
class DiceBCELoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceBCELoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()  # BCE with logits
        self.smooth = smooth

    def dice_loss(self, outputs, targets):
        outputs = torch.sigmoid(outputs)  # Convert logits to probabilities
        outputs = outputs.view(-1)
        targets = targets.view(-1)
        intersection = (outputs * targets).sum()
        return 1 - (2. * intersection + self.smooth) / (outputs.sum() + targets.sum() + self.smooth)

    def forward(self, outputs, targets):
        # If outputs is a tuple (from deep supervision), handle each output separately
        if isinstance(outputs, tuple):
            final, aux1, aux2 = outputs  # Unpack the tuple
            # Compute loss for each output
            bce_final = self.bce(final, targets)
            dice_final = self.dice_loss(final, targets)
            bce_aux1 = self.bce(aux1, targets)
            dice_aux1 = self.dice_loss(aux1, targets)
            bce_aux2 = self.bce(aux2, targets)
            dice_aux2 = self.dice_loss(aux2, targets)
            # Combine losses with weights
            total_loss = (bce_final + dice_final) + 0.2 * (bce_aux1 + dice_aux1) + 0.1 * (bce_aux2 + dice_aux2)
            return total_loss
        else:
            # For inference mode (single output)
            bce_loss = self.bce(outputs, targets)
            dice_loss = self.dice_loss(outputs, targets)
            return bce_loss + dice_loss

# Assuming model is already defined (CustomUNetWithEfficientNetB3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # Your model instance

# Initialize the loss function and optimizer
criterion = DiceBCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.5)

num_epochs = 12

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, masks in tqdm(train_loader):
        images = images.to(device)
        masks = masks.to(device).float()  # Ensure masks are float for BCE

        optimizer.zero_grad()
        outputs = model(images)  # Returns tuple (final, aux1, aux2) in training mode

        # Reshape masks to match outputs (B, 1, H, W)
        masks = masks.unsqueeze(1)  # Add channel dimension if not already present

        # Compute the combined Dice + BCE loss
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Step the scheduler at the end of each epoch
    scheduler.step()

100%|██████████| 251/251 [01:51<00:00,  2.26it/s]


Epoch [1/12], Loss: 0.6773


100%|██████████| 251/251 [01:50<00:00,  2.28it/s]


Epoch [2/12], Loss: 0.2696


100%|██████████| 251/251 [01:49<00:00,  2.28it/s]


...

Epoch [10/12], Loss: 0.1592


100%|██████████| 251/251 [01:49<00:00,  2.29it/s]


Epoch [11/12], Loss: 0.1567


100%|██████████| 251/251 [01:50<00:00,  2.28it/s]

Epoch [12/12], Loss: 0.1538


In [6]:
import torch
import numpy as np
from sklearn.metrics import jaccard_score, f1_score

def evaluate_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_iou = 0
    total_dice = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_batches = len(data_loader)

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device).float()

            outputs = model(images)
            predictions = torch.sigmoid(outputs) > 0.5  # Apply sigmoid and threshold
            
            # Convert predictions and masks to numpy arrays for metrics calculation
            predictions_np = predictions.cpu().numpy().astype(int).flatten()
            masks_np = masks.cpu().numpy().astype(int).flatten()

            # Calculate metrics
            iou = jaccard_score(masks_np, predictions_np, average='binary', zero_division=1)
            total_iou += iou

            # Calculate Dice Coefficient
            intersection = np.sum(predictions_np * masks_np)
            dice = (2. * intersection) / (np.sum(predictions_np) + np.sum(masks_np) + 1e-6)
            total_dice += dice

            # Calculate Precision, Recall, and F1 Score
            tp = np.sum(predictions_np * masks_np)
            fp = np.sum(predictions_np * (1 - masks_np))
            fn = np.sum((1 - predictions_np) * masks_np)
            
            precision = tp / (tp + fp + 1e-6)
            recall = tp / (tp + fn + 1e-6)
            f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
            
            total_precision += precision
            total_recall += recall
            total_f1 += f1

    avg_iou = total_iou / num_batches
    avg_dice = total_dice / num_batches
    avg_precision = total_precision / num_batches
    avg_recall = total_recall / num_batches
    avg_f1 = total_f1 / num_batches

    print(f'Average IoU: {avg_iou:.4f}')
    print(f'Average Dice Coefficient: {avg_dice:.4f}')
    print(f'Average Precision: {avg_precision:.4f}')
    print(f'Average Recall: {avg_recall:.4f}')
    print(f'Average F1 Score: {avg_f1:.4f}')

# Evaluate the model on the test dataset
evaluate_model(model, test_loader, device)

Average IoU: 0.8984
Average Dice Coefficient: 0.9464
Average Precision: 0.9505
Average Recall: 0.9426
Average F1 Score: 0.9464
